# STM32 flashing from rpi

* STM32 Bootloader
  * [AN2606](https://www.st.com/resource/en/application_note/cd00167594-stm32-microcontroller-system-memory-boot-mode-stmicroelectronics.pdf)
     * STM32F40xxx/41xxx (Table 53, page 109)
     * Bootloader enabled on 
        * USART1: RX=PA10, TX=PA9
        * USART3: RX=PB11, TX=PB10 or RX=PC11, TX=PC10
     * Adafruit STM32F405
        * RX/TX pins are PB11, PB10 (UART3)
        * others not available

* [MicroPython UART REPL](https://github.com/micropython/micropython/blob/master/ports/stm32/boards/NUCLEO_F411RE/mpconfigboard.h)
  * [MicroPython Forum](https://forum.micropython.org/viewtopic.php?f=6&t=7425)

* Serial flasher:
  * [Python `stm32loader` - doesn't work](https://pypi.org/project/stm32loader/)
  * [C `stm32flash`](https://sourceforge.net/p/stm32flash/wiki/Home/)
  * [Instructions (C)](https://siliconjunction.wordpress.com/2017/03/21/flashing-the-stm32f-board-using-a-raspberry-pi-3/)



## C flasher

In [ ]:
%%bash

git clone https://git.code.sf.net/p/stm32flash/code stm32flash-code
cd stm32flash-code
sudo make install

In [ ]:
!stm32flash -h

In [1]:
!stm32flash /dev/serial0

stm32flash 0.6

http://stm32flash.sourceforge.net/

Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB



In [3]:
!stm32flash -v -S 0x08000000 -w $IOT/iot-bin/micropython/stm32-MOTOR_HAT_BREAKOUT-firmware0.bin /dev/serial0 

stm32flash 0.6

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB
Write to memory
Erasing memory
Wrote and verified address 0x080039c4 (100.00%) Done.



In [4]:
!stm32flash -v -S 0x08020000 -w $IOT/iot-bin/micropython/stm32-MOTOR_HAT_BREAKOUT-firmware1.bin /dev/serial0 

stm32flash 0.6

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB
Write to memory
Erasing memory
Wrote and verified address 0x08073464 (100.00%) Done.



In [6]:
import serial, time

with serial.Serial("/dev/serial0", baudrate=115200, timeout=1.0) as dev:
    for i in range(5000):
        # dev.write(b"print('i = {}'.format(i))\n\r")
        rcv = dev.read().decode()
        print(rcv, end='')

MPY: can't mount SD card
1 chips detected. Total flash size 2MiB.
MicroPython v1.13-361-g0ed11821d0-dirty on 2021-03-09; Motor Breakout Hat with STM32F405RG
Type "help()" for more information.
>>> 
MPY: can't mount SD card
1 chips detected. Total flash size 2MiB.
MicroPython v1.13-361-g0ed11821d0-dirty on 2021-03-09; Motor Breakout Hat with STM32F405RG
Type "help()" for more information.
>>> 
> >>
r P;T-  i


KeyboardInterrupt: 